In [ ]:
import os,subprocess,struct,gzip
from pylab import *
from collections import defaultdict

result     = subprocess.run(['find','/usr/share/consolefonts/','-iname','*.psf*'],stdout=subprocess.PIPE,stderr=subprocess.DEVNULL)
fontfiles  = result.stdout.split()
compressed = [f for f in fontfiles if f.lower()[-7:]==b'.psf.gz']

In [ ]:
all_supported = {}

for f in compressed:
    with gzip.open(f,'rb') as gf: b = b''.join(gf)

    is_psf1 = struct.unpack('BB',b[:2])==(0x36,0x04)
    is_psf2 = False if is_psf1 else struct.unpack('BBBB',b[:4])==(0x72,0xb5,0x4a,0x86)
    if not (is_psf1 or is_psf2):
        raise ValueError('Magic number does not match either psf1 or psf2 format')
    if is_psf2:
        #print('Skipping psf2 file',f)
        continue
    if is_psf1:
        magic, mode, charsize = struct.unpack('2sBB',b[:4])
        PSF1_MODE512    =0x01
        PSF1_MODEHASTAB =0x02
        PSF1_MODEHASSEQ =0x04
        PSF1_MAXMODE    =0x05
        PSF1_SEPARATOR  =0xFFFF
        PSF1_STARTSEQ   =0xFFFE

        is512  = (mode&PSF1_MODE512)>0
        hastab = (mode&PSF1_MODEHASTAB)>0
        hasseq = (mode&PSF1_MODEHASSEQ)>0
        height = charsize
        width  = 8
        nchars = 512 if is512 else 256
        #print(is512, hastab, hasseq, width, height, nchars)

        bitsperchar  = width*height
        bytesperchar = bitsperchar//8
        fontbytes    = bytesperchar*nchars
        filedata     = b[4:]
        fontdata     = filedata[:fontbytes]
        unicodeinfo  = filedata[fontbytes:]

        nshorts = len(unicodeinfo)//2
        codes   = struct.unpack('<'+'H'*nshorts, unicodeinfo)

        # Combining accents will break this
        groups    = ''.join([chr(c) for c in codes]).split(chr(0xFFFF))
        supported = ''.join(groups[:nchars])
        all_supported[f] = {*supported}

print(''.join(sorted(list(set.intersection(*all_supported.values())))))

 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ §–•←↑→↓−▲▶▼◀♦


In [93]:
import unicodedata

# sort by frequency
counter = defaultdict(lambda:0)
for f,chrs in all_supported.items():
    for c in chrs:
        counter[c]+=1
        
# Also add in these characters scraped from historical code pages
d = open('./fontdescription/codepagedump','r').readlines()
for c in ''.join(d):
    counter[c]+=1
        
keys    = sorted([*counter.keys()])
order   = argsort(-array([counter[k] for k in keys]))

maybe_support = set()
for  k in array(keys)[order]:
    if k in '﷽': continue
    try:
        n = unicodedata.name(k)
    except:
        print('unicode data doesn\'t know the name of',k)
    if 'ETHIOPIC' in n: continue
    if 'HEBREW' in n: continue
    if 'ARABIC' in n: continue
    d = unicodedata.bidirectional(k)
    if d in ('R','AL','RLE','RLO','RLI'): continue # no right-to-left
    #print('    '+k+'% 3d%%'%(100*counter[k]/len(all_supported)))
    # Skip combining
    c = unicodedata.category(k)
    if c[0]=='M':
        continue
    maybe_support.add(k)

unicode data doesn't know the name of 

unicode data doesn't know the name of ﷾
unicode data doesn't know the name of ﷿


In [94]:

maybe_support -= {*'\n\r\t'}

In [95]:
j=0
for i in sorted(list({ord(c) for c in maybe_support})):
    print(chr(i),end='')
    if j%40==39:
        print('')
    j+=1

 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFG
HIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmno
pqrstuvwxyz{|}~ ¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸
¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßà
áâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿĀāĂăĄąĆćĈ
ĉĊċČčĎďĐđĒēĔĕĖėĘęĚěĜĝĞğĠġĢģĤĥĦħĨĩĪīĮįİıĲ
ĳĴĵĶķĸĹĺĻļĽľĿŀŁłŃńŅņŇňŉŊŋŌōŎŏŐőŒœŔŕŖŗŘřŚ
śŜŝŞşŠšŢţŤťŦŧŨũŪūŬŭŮůŰűŲųŴŵŶŷŸŹźŻżŽžſƒƠơ
ƯưƷǅǈǋǎǔǞǟǠǡǢǣǤǥǦǧǨǩǪǫǬǭǮǯǴǵǸǹǾǿȘșȚțȞȟɑɒ
ɔəɛɜɡɪʃʊʌʒʟʲʼʽʾˆˇˈˉˋˌː˘˙˚˛˜˝ˮͺ΄΅ΆΈΉΊΌΎΏΐ
ΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩΪΫάέήίΰαβγδεζηθι
κλμνξοπρςστυφχψωϊϋόύώϑϒϕϖϳϴЀЁЂЃЄЅІЇЈЉЊЋЌ
ЍЎЏАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгд
ежзийклмнопрстуфхцчшщъыьэюяѐёђѓєѕіїјљњћќ
ѝўџҊҋҌҍҎҏҐґҒғҔҕҖҗҘҙҚқҜҝҞҟҠҡҢңҤҥҦҧҨҩҪҫҬҭҮ
үҰұҲҳҴҵҶҷҸҹҺһҼҽҾҿӅӆӇӈӉӊӍӎӐӑӒӓӖӗӘәӜӝӞӟӢӣӤ
ӥӦӧӨөӬӭӮӯӰӱӲӳӴӵӸӹԱԲԳԴԵԶԷԸԹԺԻԼԽԾԿՀՁՂՃՄՅՆՇ
ՈՉՊՋՌՍՎՏՐՑՒՓՔՕՖ՚՛՜՝՞աբգդեզէըթժիլխծկհձղճմ
յնշոչպջռսվտրցւփքօֆև։֊अआइईउऊऋऌऍऎएऐऑऒओऔकखग
घङचछजझञटठडढणतथदधनऩपफबभमयरऱलळऴवशषसहऽॐॠॡ।०
१२३४५६७८९বলਕਜਪਬகകമയലളกขฃคฅฆงจฉชซฌญฎฏฐฑฒณ
ดตถทธนบปผฝพฟภมยรฤลฦวศษสหฬอฮฯะาำ฿เแโใไๅๆ๏
๐๑๒๓๔๕๖๗๘๙๚๛ກຂຄງຈຊຍດຕຖທນບປຜຝພຟມຢຣລວສຫອຮຯ
ະາຳຽເແໂໃໄໆ໐໑໒໓໔໕໖໗໘໙ໜໝაბგდევზთიკლმნოპჟრს
ტუფქღყშჩცძწჭხჯჰჱ